In [7]:
import numpy as np
from scipy.sparse import load_npz
import pickle
from queue import PriorityQueue
import math

# csr_heuristic_matrix = load_npz('heuristic_matrix.npz')  # Uncomment if you have a heuristic matrix


#Calculate haversine distance
def haversine(coord1, coord2):
    R = 6371  # Radius of the Earth in kilometers
    lat1, lon1 = math.radians(coord1[0]), math.radians(coord1[1])
    lat2, lon2 = math.radians(coord2[0]), math.radians(coord2[1])

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R * c

def find_nearest_matrix_index(input_coord, index_to_coord_map):
    """Find the nearest point's index in the matrix to the input_coord."""
    min_distance = float('inf')
    nearest_index = None

    for index, coord in index_to_coord_map.items():
        distance = haversine(input_coord, coord)
        if distance < min_distance:
            min_distance = distance
            nearest_index = index

    return nearest_index



In [8]:
from scipy.sparse import lil_matrix
import xml.etree.ElementTree as ET

# Parse the OSM file
tree = ET.parse(r"C:\Users\FPTSHOP\Downloads\map (2).osm") # Replace with your file path
root = tree.getroot()

# Extract nodes and ways
nodes = {}
ways = []

for element in root:
    if element.tag == 'node':
        node_id = int(element.attrib['id'])
        nodes[node_id] = len(nodes)
    elif element.tag == 'way':
        nd_refs = [int(nd.attrib['ref']) for nd in element if nd.tag == 'nd']
        ways.append(nd_refs)

# Number of nodes
num_nodes = len(nodes)

# Create the sparse adjacency matrix
sparse_adjacency_matrix = lil_matrix((num_nodes, num_nodes), dtype=int)

# Populate the sparse matrix
for way in ways:
    for i in range(len(way) - 1):
        if way[i] in nodes and way[i + 1] in nodes:
            idx1, idx2 = nodes[way[i]], nodes[way[i + 1]]
            sparse_adjacency_matrix[idx1, idx2] = 1
            sparse_adjacency_matrix[idx2, idx1] = 1  # Assuming bidirectional roads

# Convert to a more efficient format (CSR)
sparse_adjacency_matrix = sparse_adjacency_matrix.tocsr()

# Display some information about the sparse matrix for verification
num_nonzero_entries = sparse_adjacency_matrix.count_nonzero()
matrix_size = sparse_adjacency_matrix.shape
(num_nonzero_entries, matrix_size)
# Assuming `nodes` is a dictionary with node IDs as keys and (lat, lon) tuples as values
node_ids = list(nodes.keys())
coords = [nodes[node_id] for node_id in node_ids]

# Creating coord_to_index and index_to_coord
coord_to_index = {coords[i]: i for i in range(len(coords))}
index_to_coord = {i: coords[i] for i in range(len(coords))}


In [9]:
from queue import PriorityQueue

def a_star(start_coord, goal_coord, sparse_adjacency_matrix, coord_to_index, index_to_coord):
    start = coord_to_index[start_coord]
    goal = coord_to_index[goal_coord]

    open_set = PriorityQueue()
    open_set.put((0, start))

    came_from = {start: None}
    cost_so_far = {start: 0}

    node_seen_count = {}
    explored_routes = []

    while not open_set.empty():
        current = open_set.get()[1]
        node_seen_count[current] = node_seen_count.get(current, 0) + 1

        if current == goal:
            break

        current_coord = index_to_coord[current]

        # Access only neighbors of the current node
        for next_node in sparse_adjacency_matrix[current].nonzero()[1]:
            next_node_coord = index_to_coord[next_node]

            # Calculate distance using Haversine formula
            distance = haversine(current_coord, next_node_coord)
            new_cost = cost_so_far[current] + distance

            if next_node not in cost_so_far or new_cost < cost_so_far[next_node]:
                cost_so_far[next_node] = new_cost
                priority = new_cost + haversine(next_node_coord, goal_coord)
                open_set.put((priority, next_node))
                came_from[next_node] = current
                explored_routes.append((current, next_node))

    # Reconstruct path
    path = []
    current = goal
    while current != start:
        if current not in came_from:
            break
        path.append(index_to_coord[current])
        current = came_from[current]
    path.append(index_to_coord[start])
    path.reverse()

    return path, cost_so_far.get(goal, float('inf')), node_seen_count, explored_routes


In [11]:
unique_nodes = set()  # Initialize a set to store unique node indices

# Iterate through the rows and columns of the sparse adjacency matrix
for row in range(sparse_adjacency_matrix.shape[0]):
    for col in sparse_adjacency_matrix[row].nonzero()[1]:
        # Add both row and column indices to the set of unique nodes
        unique_nodes.add(row)
        unique_nodes.add(col)

# Convert the set of unique node indices to a list
unique_nodes_list = list(unique_nodes)

# If needed, you can find the corresponding coordinates using index_to_coord
unique_node_coordinates = [index_to_coord[node_index] for node_index in unique_nodes_list]


In [14]:
# Example usage
memory_limit = 100
start_coord = (21.0216642, 105.8388283)
goal_coord =( 21.0081752,105.8456702)
path, total_cost, node_seen_count, explored_routes = a_star(1, 20,sparse_adjacency_matrix,coord_to_index,index_to_coord)

print("Path:", path)
print("Total Cost:", total_cost)
print("Node Seen Count:", node_seen_count)
print("Explored Routes:", explored_routes)

TypeError: 'int' object is not subscriptable

In [ ]:
import folium

def plot_search_visualization(path, node_seen_count, explored_routes, index_to_coord):
    if not path:
        raise ValueError("Path is empty, cannot visualize.")

    # Initialize the map centered around the start of the path
    m = folium.Map(location=path[0], zoom_start=14)

    # Add markers for the start and end of the path
    folium.Marker(path[0], popup='Start', icon=folium.Icon(color='green')).add_to(m)
    folium.Marker(path[-1], popup='Goal', icon=folium.Icon(color='red')).add_to(m)

    # Add the true path
    folium.PolyLine(path, color='blue', weight=2.5, opacity=1).add_to(m)

    # Add markers for nodes seen
    for node, count in node_seen_count.items():
        if node in index_to_coord:
            coord = index_to_coord[node]
            folium.CircleMarker(coord, radius=5, color='orange', fill=True, fill_color='orange', popup=f'Seen {count} times').add_to(m)

    # Add lines for explored routes
    for route in explored_routes:
        start_node, end_node = route
        if start_node in index_to_coord and end_node in index_to_coord:
            start_coord = index_to_coord[start_node]
            end_coord = index_to_coord[end_node]
            folium.PolyLine([start_coord, end_coord], color='grey', weight=1, opacity=0.5).add_to(m)

    return m

# Example usage
# Assuming you have index_to_coord mapping and the other required data from your A* output
map_visualization = plot_search_visualization(path, node_seen_count, explored_routes, index_to_coord)
map_visualization.save('search_visualization.html')
